- Nama            : Rizqi Amalia Kartika
- Cohort ID       : MC219D5X1732
- Group Belajar   : MC-38

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer
from gensim.models import Word2Vec
from sklearn.svm import SVC
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rizqiamaliakartika/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rizqiamaliakartika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("jago_reviews.csv")
print(df.columns)
df.head()

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,a6c9d785-bf3d-478a-a215-29a08b74d45d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,4,0,NaN,2025-04-13 21:00:50,"Hai, Jagoan! Terima kasih sudah memberikan ula...",2025-04-13 22:37:48,NaN
1,a06c6e14-f57c-4ca5-82a8-3a98cdb44a29,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kantongnya berguna sekali. sayangnya utk qris ...,5,0,8.62.3,2025-04-13 20:29:45,NaN,NaN,8.62.3
2,9c82e781-14dd-45c1-a2ea-a57de3a437ea,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"love banget sama bank digital 1 ini,, untuk me...",5,0,8.62.3,2025-04-13 19:57:39,NaN,NaN,8.62.3
3,1a600c0d-7f09-4cd3-b6c3-bd53a235f1b6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,baik,5,0,8.62.2,2025-04-13 19:32:54,NaN,NaN,8.62.2
4,29026806-cc65-452e-8de4-dfb0837b0692,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,oke bgtt ui simpel dan ada kantong buat pisahi...,4,0,8.62.3,2025-04-13 19:04:22,"Hai, Jagoan! Terima kasih sudah memberikan ula...",2025-04-14 03:18:39,8.62.3


In [3]:
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

df['label'] = df['score'].apply(label_sentiment)
print(df['label'].value_counts())

label
positif    6906
negatif    2716
netral      378
Name: count, dtype: int64


## Skema 1 – SVM + TF-IDF

In [4]:
import re

stop_words = set(stopwords.words('indonesian'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # hapus simbol/angka
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['cleaned'] = df['content'].astype(str).apply(clean_text)

In [5]:
df[['content', 'cleaned']].sample(5)

,content,cleaned
6321,Gak bisa masuk padahal dah tau pasword masih a...,gak masuk dah tau pasword aja gak masuk
206,sengaja saya kasih download supaya bisa kasih ...,sengaja kasih download kasih review tau mengat...
7586,Haloo. Terimakasii sudah membuat apk jago syar...,haloo terimakasii apk jago syariah membantu ba...
693,terbaik memang,terbaik
3635,bagus,bagus


In [6]:
from sklearn.model_selection import train_test_split

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(df['cleaned'], df['label'], test_size=0.2, random_state=42)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Menggunakan TF-IDF untuk ekstraksi fitur
# vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=3)

In [8]:
# Transformasi data latih dan data uji
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [9]:
from sklearn.svm import LinearSVC

# Latih model
svm_model = LinearSVC()
svm_model.fit(X_train_vec, y_train)

# Prediksi
y_pred = svm_model.predict(X_test_vec)

# Evaluasi
from sklearn.metrics import classification_report, accuracy_score

print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 0.874
Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.81      0.81      0.81       528
      netral       0.30      0.04      0.07        80
     positif       0.90      0.95      0.92      1392

    accuracy                           0.87      2000
   macro avg       0.67      0.60      0.60      2000
weighted avg       0.85      0.87      0.86      2000



/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


## Skema 2 – LSTM + Tokenizer

In [10]:
# Tokenisasi
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences
max_length = 100  # bisa disesuaikan
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Encode label (string ke angka)
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

# Untuk LSTM, ubah label ke categorical (one-hot)
y_train_cat = tf.keras.utils.to_categorical(y_train_enc)
y_test_cat = tf.keras.utils.to_categorical(y_test_enc)

In [11]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_length))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 kelas: positif, netral, negatif

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training model
history = model.fit(X_train_pad, y_train_cat, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test_cat))

/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 71ms/step - accuracy: 0.6797 - loss: 0.7822 - val_accuracy: 0.6960 - val_loss: 0.7331
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - accuracy: 0.6927 - loss: 0.7353 - val_accuracy: 0.6960 - val_loss: 0.7465
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.6842 - loss: 0.7464 - val_accuracy: 0.6960 - val_loss: 0.7332
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 82ms/step - accuracy: 0.6782 - loss: 0.7611 - val_accuracy: 0.6960 - val_loss: 0.7328
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.6915 - loss: 0.7331 - val_accuracy: 0.6960 - val_loss: 0.7334


In [12]:
# Evaluasi akhir
loss, acc = model.evaluate(X_test_pad, y_test_cat)
print(f"Accuracy on testing set: {acc * 100:.2f}%")

# Prediksi label kategorikal
y_pred_prob = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['negatif', 'netral', 'positif']))

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6935 - loss: 0.7314
Accuracy on testing set: 69.60%
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step
Classification Report:
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00       528
      netral       0.00      0.00      0.00        80
     positif       0.70      1.00      0.82      1392

    accuracy                           0.70      2000
   macro avg       0.23      0.33      0.27      2000
weighted avg       0.48      0.70      0.57      2000



/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

## Skema 3 – Logistic Regression + TF-IDF

In [13]:
from sklearn.linear_model import LogisticRegression

# Inisialisasi model
logreg = LogisticRegression(max_iter=1000, random_state=42)

# Training model
logreg.fit(X_train_vec, y_train)

# Prediksi
y_pred_logreg = logreg.predict(X_test_vec)

# Evaluasi
from sklearn.metrics import accuracy_score, classification_report

print("Akurasi:", accuracy_score(y_test, y_pred_logreg))
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred_logreg))


Akurasi: 0.8815
Laporan Klasifikasi:


/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     negatif       0.81      0.82      0.82       528
      netral       0.00      0.00      0.00        80
     positif       0.91      0.95      0.93      1392

    accuracy                           0.88      2000
   macro avg       0.57      0.59      0.58      2000
weighted avg       0.85      0.88      0.86      2000



/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rizqiamaliakartika/PYTHON/studysession/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
# Contoh inference menggunakan Logistic Regression

def infer_logreg(text):
    # Preprocessing
    text_cleaned = clean_text(text)
    vectorized = vectorizer.transform([text_cleaned])

    # Prediksi
    pred = logreg.predict(vectorized)
    print(f"Teks: {text}")
    print(f"Prediksi Sentimen: {pred[0]}")

infer_logreg("Jelek banget")

Teks: Jelek banget
Prediksi Sentimen: negatif
